### Practice Answer

* **综合题：下面是4个产品(p1, p2, p3, p4)，每个产品对应一组数据，其中第1个整数型数据表示库存量，第2，3，4个数据为浮点型，表示产品在三个不同标准下的评分，第5个数据表示更新状态，True表示继续更新，False表示停止更新，第6个数据格式是string，包含了评价信息。**  
**问题1**：对于第一个数据，产品1，2需要每3个进行打包，产品3，4则需要每2个进行打包，问：每个产品需要补充多少库存才能确保打包完成后，仓库中不留产品。  
**问题2**：在三个标准下的评分由于录入数据的疏忽，出现了原本应为82.3的分数变为了'8 2.3'，请恢复这些数据，并把值为None的数据用其它有分数数据的平均值进行填补。  
**问题3**：对于第5个数据的更新状态，请将True变为1，False变为0  
**问题4**：请删去第6个数据评论文字中的非英文与数字部分，并将所有英文转化为小写，将Xm3 形式的字符串中的 X 提取出来，变为浮点型  
如：'Good product and 30m^3 very popuLar but hard to use!😊' 变为 ['good', 'product', 'and', 'very', 'popular', 'but', 'hard', 'to', 'use', 30.0]  


In [2]:
data = {'p1': [32, 98.2, 99.1, '97. 0', True, 'Good product and 30m^3 very popuLar but hard to use!😊'], 
'p2': (66, None, 96.0, 95.4, True, 'Good product with 10m^3 solid quality!'), 
'p3': (73, 90.0, '8 2.3', None, True, 'Fair 5m^3 qualiTy...'), 
'p4': (14, '9 9.0', None, None, False, 'Excellent and 17m^3 easy to use!!')}

### Question 1

In [167]:
# code 1
def stock(stocknumber,index):
    if stocknumber%index:
        add_data = index - stocknumber%index
    else:
        add_data = 0
    return add_data
question1_p1 = stock(data['p1'][0],3)
question1_p2 = stock(data['p2'][0],3)
question1_p3 = stock(data['p3'][0],2)
question1_p4 = stock(data['p4'][0],2)

for i, j in enumerate(['question1_p' + str(i) for i in range(1, 5)]):
    print('p{0}产品需要补充库存：3i + {1}'.format(i, eval(j)))

p0产品需要补充库存：3i + 1
p1产品需要补充库存：3i + 0
p2产品需要补充库存：3i + 1
p3产品需要补充库存：3i + 0


In [168]:
# code 2
inventory = [each[0] for each in list(data.values())]
result = list(map(lambda x: '3i + '+str(3 - x%3) if x%3 != 0 else '3i', inventory[0:2])) + \
         list(map(lambda x: '2i + '+str(2 - x%2) if x%2 != 0 else '2i', inventory[2:]))
# print
print('result 1:\n')
for i in range(len(result)):
    print('PRODUCT {0}: {1}'.format(i+1, result[i]))

result 1:

PRODUCT 1: 3i + 1
PRODUCT 2: 3i
PRODUCT 3: 2i + 1
PRODUCT 4: 2i


In [169]:
# code 3
def smod(x, mul):
    if x%mul != 0: return '{0}i + {1}'.format(mul, 3 - x%mul)
    else: return '{0}i'.format(mul)
from functools import partial
f2 = partial(smod, mul = 2)
f3 = partial(smod, mul = 3)
result = list(map(f3, inventory[0:2])) + \
         list(map(f2, inventory[2:]))
# print
print('result 2:\n')
for i in range(len(result)):
    print('PRODUCT {0}: {1}'.format(i+1, result[i]))

result 2:

PRODUCT 1: 3i + 1
PRODUCT 2: 3i
PRODUCT 3: 2i + 2
PRODUCT 4: 2i


### Question 2

In [170]:
# code 1
# fix score & convert None to 0
score = [each[1:4] for each in data.values()]
new_score = [list(map(lambda x: float(''.join(x.split(' '))) if type(x) == str else 0 if x == None else x , each_scorer)) for each_scorer in score]
none_index = [[i for i, each in enumerate(each_) if each == None] for each_ in score]
compensate_score = [sum(each_score)/(len(each_score) - len(each_index)) for each_score, each_index in zip(new_score, none_index)]
for i, score_index in enumerate(zip(compensate_score, none_index)):
    # fix 0 score
    if len(score_index[1]) != 0:
        for j in score_index[1]:
            new_score[i][j] = score_index[0]
print(new_score)

[[98.2, 99.1, 97.0], [95.7, 96.0, 95.4], [90.0, 82.3, 86.15], [99.0, 99.0, 99.0]]


In [171]:
# code 2
def cleanStr(str_iter):
    '''function: clean str; example: "98. 32"'''
    return list(map(lambda x: float(''.join(x.split(' '))) if type(x) == str else x, str_iter))
def mapNone(none_iter):
    '''function: map None to 0;'''
    return list(map(lambda x: 0 if x == None else x, none_iter))
def superIndex(data_iter, target):
    '''find all None values index and replace with target'''
    return [i for i, each in enumerate(data_iter) if each == target]
def compensate(data_iter, comp_index, comp_num):
    '''data_iter: original data; comp_index: compensate index of data; comp_num: compensate number of data'''
    for i, score_index in enumerate(zip(comp_num, comp_index)):
        if len(score_index[1]) != 0:
            for j in score_index[1]:
                data_iter[i][j] = score_index[0]
    return data_iter
# extract data
score = [each[1:4] for each in data.values()]
# map str
new_score = [cleanStr(each) for each in score]
# map None
new_score = [mapNone(each) for each in new_score]
# super index
super_index = [superIndex(each, None) for each in score]
# compensate
compensate_score = [sum(each_score)/(len(each_score) - len(each_index)) for each_score, each_index in zip(new_score, super_index)]
compensate(new_score, super_index, compensate_score)

[[98.2, 99.1, 97.0],
 [95.7, 96.0, 95.4],
 [90.0, 82.3, 86.15],
 [99.0, 99.0, 99.0]]

### Question 3

In [61]:
int(True)

1

In [172]:
# Question 3
update = [each[4] for each in data.values()]
print('Transform', list(map(lambda x: 1 if x == True else 0 if x == False else x, update)))
print('Transform', list(map(int, update)))

Transform [1, 1, 1, 0]
Transform [1, 1, 1, 0]


In [173]:
cdata = data.copy()
for key in cdata.keys():
    cdata[key] = list(cdata[key])
    cdata[key][4] = int(cdata[key][4])
print(cdata)

{'p1': [32, 98.2, 99.1, '97. 0', 1, 'Good product and 30m^3 very popuLar but hard to use!😊'], 'p2': [66, None, 96.0, 95.4, 1, 'Good product with 10m^3 solid quality!'], 'p3': [73, 90.0, '8 2.3', None, 1, 'Fair 5m^3 qualiTy...'], 'p4': [14, '9 9.0', None, None, 0, 'Excellent and 17m^3 easy to use!!']}


### Question 4

In [56]:
data = {'p1': [32, 98.2, 99.1, '97. 0', True, 'Good product and 30m^3 very popuLar but hard to use!😊'], 
        'p2': (66, None, 96.0, 95.4, True, 'Good product with 10m^3 solid quality!'), 
        'p3': (73, 90.0, '8 2.3', None, True, 'Fair 5m^3 qualiTy...'), 
        'p4': (14, '9 9.0', None, None, False, 'Excellent and 17m^3 easy to use!!')}

In [58]:
# code 1
def cleanWord(sen):
    return ''.join(list(filter(lambda ch: ch in 'abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ123456789', sen.lower())))
txt = [each[5] for each in data.values()]
words = [cleanWord(each).split(' ') for each in txt]
bool_words = [list(map(lambda x: x.isalpha(), each)) for each in words]
# method 1
new_words = [list(filter(lambda x: j[words[i].index(x)],words[i])) for i,j in enumerate(bool_words)]
nums = [list(filter(lambda x: not j[words[i].index(x)],words[i])) for i,j in enumerate(bool_words)]
new_nums = [list(map(lambda x: int(x[:-2]), each)) for each in nums]

new_txt = []
for i, j in zip(new_words, new_nums):
    new_txt.append(i+j)
print('Result: \n',new_txt)

Result: 
 [['good', 'product', 'and', 'very', 'popular', 'but', 'hard', 'to', 'use', 3], ['good', 'product', 'with', 'solid', 'quality', 1], ['fair', 'quality', 5], ['excellent', 'and', 'easy', 'to', 'use', 17]]


In [60]:
# code 2
from itertools import compress
Tlist = [1, 2, 3]
Tbool = [True, False, True]
NTList = list(compress(Tlist, Tbool))
NTList

[1, 3]

In [45]:
# code 3
# re module
import re
for key in data.keys():
    data[key] = list(data[key])
    data[key][5] = re.sub("[^A-Za-z0-9 ]", "", data[key][5]).split(" ")
    for i, each in enumerate(data[key][5]):
        temp = re.match("[0-9]*m3", each)
        if temp:
            temp = temp.group()
            ret = temp.rstrip('m3')
            data[key][5].append(float(ret))
            data[key][5].pop(i)
            break
print(data)

{'p1': [32, 98.2, 99.1, '97. 0', True, ['Good', 'product', 'and', 'very', 'popuLar', 'but', 'hard', 'to', 'use', 30.0]], 'p2': [66, None, 96.0, 95.4, True, ['Good', 'product', 'with', 'solid', 'quality', 10.0]], 'p3': [73, 90.0, '8 2.3', None, True, ['Fair', 'qualiTy', 5.0]], 'p4': [14, '9 9.0', None, None, False, ['Excellent', 'and', 'easy', 'to', 'use', 17.0]]}
